In [ ]:
import numpy as np
import pandas as pd
from scipy import stats
import seaborn as sn
from collections import Counter
import matplotlib.pyplot as plt
import os
import random
import scipy
from itertools import chain
from absl import logging
import tensorflow as tf
import tensorflow_hub as hub
import re
import warnings
import math
from sklearn.exceptions import ConvergenceWarning

module_url = "https://tfhub.dev/google/universal-sentence-encoder/4"
model = hub.load(module_url)
print ("module %s loaded" % module_url)
def embed(input):
  return model(input)
os.chdir('/gpfs/milgram/project/chun/jk2992/rest_thoughts/') # change to your folder path

module https://tfhub.dev/google/universal-sentence-encoder/4 loaded


In [2]:
def conv_r2z(r):
    with np.errstate(invalid='ignore', divide='ignore'):
        return 0.5 * (np.log(1 + r) - np.log(1 - r))
def conv_z2r(z):
    with np.errstate(invalid='ignore', divide='ignore'):
        return (np.exp(2 * z) - 1) / (np.exp(2 * z) + 1)
def get_info(name):
    tmp = name.split('_')
    sub = tmp[0]
    ses = tmp[1]
    return sub, ses
def get_num(string):
    num = []
    for i in range(len(string)):
        if i % 5 == 2:
            num.append(int(string[i]))
    return num
def reshape_FC(fc):
    fc = np.transpose(fc,(2,0,1))
    fc = np.reshape(fc,(fc.shape[0],fc.shape[1]*fc.shape[2]))
    return fc
def nancorr(tc1,tc2):
    # get nan position
    nanidx1 = np.where(np.isnan(tc1))
    nanidx2 = np.where(np.isnan(tc2))
    nanidx = np.union1d(nanidx1,nanidx2)
    tc1 = np.delete(tc1,nanidx)
    tc2 = np.delete(tc2,nanidx)
    # run correlation
    rval = stats.pearsonr(tc1,tc2)[0]
    return rval
def cosine_similarity(vector1, vector2):
    dot_product = np.dot(vector1, vector2)
    magnitude_vector1 = np.linalg.norm(vector1)
    magnitude_vector2 = np.linalg.norm(vector2)
    return dot_product / (magnitude_vector1 * magnitude_vector2)
def onetail_p(real, null):
    p = (1 + np.sum(null>=real)) / (1+len(null))
    print(str(np.sum(null>=real))+' among '+str(len(null))+' null has higher r value than actual prediction')
    return p
def nanspearmanr(tc1,tc2):
    # get nan position
    nanidx1 = np.where(np.isnan(tc1))
    nanidx2 = np.where(np.isnan(tc2))
    nanidx = np.union1d(nanidx1,nanidx2)
    tc1 = np.delete(tc1,nanidx)
    tc2 = np.delete(tc2,nanidx)
    # run correlation
    rval = stats.spearmanr(tc1,tc2)[0]
    return rval

# thought content

In [3]:
nsubj = 60
df_transcripts = pd.read_csv('./data/beh/thoughts_transcripts.csv') 
# the raw recording transcriptions will not be publicly available due to privacy concerns
# however, we shared the embeddings of these transcripts
sub_id = np.unique(df_transcripts['subid'])

## asking if people with similar thoughts have similar FC profiles
all_sub_embed = [] # first calculate the embeddings of overall thoughts (concatenated across trials)
for sub in sub_id:
    # print('Running '+str(sub))
    thought_bysub = df_transcripts[df_transcripts['subid']==sub]['Transcriptions']
    concate_bysub = "" 
    for trial in range(thought_bysub.index[0],thought_bysub.index[len(thought_bysub.index)-1]+1):
        if pd.isna(thought_bysub[trial]):
            pass
        else:
            concate_bysub = concate_bysub + '. ' + thought_bysub[trial]
    concate_bysub = [concate_bysub]
    this_sub_embed = embed(concate_bysub)
    this_sub_embed = np.squeeze(this_sub_embed)
    all_sub_embed.append(this_sub_embed)
scipy.io.savemat('./data/beh/thought_embeddings.mat',{'embeddings':all_sub_embed})

embed_sim = [] # calculate between subject similarity
for i in range(nsubj-1):
    for j in range(i+1,nsubj):
        tmp_r = cosine_similarity(all_sub_embed[i],all_sub_embed[j])
        embed_sim.append(tmp_r)
embed_sim = np.asarray(embed_sim)

In [8]:
# read in brain data
rest_fc = scipy.io.loadmat('./data/brain/rest_fc.mat')['rest'][0]

# calculate FC by subject
nsubj = len(rest_fc)
fc_bysub = []
for i in range(nsubj):
    fc_bysub.append(reshape_FC(rest_fc[i]))

fingerprint_bysub = []
for sub in range(nsubj):
    this_sub = np.nanmean(fc_bysub[sub],axis=1) # calculate the mean FC profile for each individual
    fingerprint_bysub.append(this_sub)

FP_sim = []
for i in range(nsubj-1):
    for j in range(i+1,nsubj):
        tmp_r = nancorr(fingerprint_bysub[i],fingerprint_bysub[j])
        FP_sim.append(tmp_r)
FP_sim = np.asarray(FP_sim)

/var/folders/dq/rwvdlsl16q96bmby7428s3700000gn/T/ipykernel_4510/783710020.py:12: RuntimeWarning: Mean of empty slice
  this_sub = np.nanmean(fc_bysub[sub],axis=1) # calculate the mean FC profile for each individual


In [9]:
actual_r = nanspearmanr(FP_sim,embed_sim)
null_r = []
niteration = 10000
for iter in range(niteration):
    random.shuffle(embed_sim)
    tmp_r = nanspearmanr(FP_sim,embed_sim)
    null_r.append(tmp_r)
null_r = np.asarray(null_r)
p = onetail_p(actual_r,null_r)
print('r = ',actual_r,'p = ',p)

scipy.io.savemat('./results/RSA/content_fingerprints.mat',{'actual_r':actual_r,'null_r':null_r}) # save the results for plotting

0 among 10000 null has higher r value than actual prediction
r =  0.09619635073154363 p =  9.999000099990002e-05


In [5]:
# asking if FC dynamics also track thought fluctuations

# calculate the embeddings of each trial
embed_bysub = []
for sub in sub_id:
    thought_bysub = df_transcripts[df_transcripts['subid']==sub]['Transcriptions']
    embed_sub = []
    for trial in range(thought_bysub.index[0],thought_bysub.index[len(thought_bysub.index)-1]+1):
        if pd.isna(thought_bysub[trial]):
            embed_sub.append(np.full(512, np.nan))
        else:
            embed_sub.append(np.squeeze(embed([thought_bysub[trial]])))
    embed_bysub.append(np.array(embed_sub))

# remove sub-1044 ses1 run2 to match brain data (convertion error)
embed_bysub[41] = np.delete(embed_bysub[41],range(8,16),axis=0)

thought_sim = [] # calculate within subject between trial similarity
for i in range(nsubj):
    r_sub = []
    for j in range(embed_bysub[i].shape[0]-1):
        for k in range(j+1,embed_bysub[i].shape[0]):
            tmp_r = cosine_similarity(embed_bysub[i][j,:],embed_bysub[i][k,:])
            r_sub.append(tmp_r)
    thought_sim.append(np.array(r_sub))

In [6]:
# calculate FC similarity within subject
fc_sim = []
for i in range(nsubj):
    r_sub = []
    for j in range(fc_bysub[i].shape[1]-1):
        for k in range(j+1,fc_bysub[i].shape[1]):
            if np.sum(np.isnan(fc_bysub[i][:,j])) == 35778 or np.sum(np.isnan(fc_bysub[i][:,k])) == 35778:
                tmp_r = np.nan
            else:
                tmp_r = nancorr(fc_bysub[i][:,j],fc_bysub[i][:,k])
            r_sub.append(tmp_r)
    fc_sim.append(np.array(r_sub))

rvals = []
for sub in range(nsubj):
    if np.all(np.isnan(thought_sim[sub])):
        r_sub = np.nan
    else:
        r_sub = nanspearmanr(fc_sim[sub],thought_sim[sub])
    rvals.append(r_sub)

In [9]:
actual_r = conv_z2r(np.nanmean(conv_r2z(np.array(rvals))))
null_r = []
niteration = 10000
for iter in range(niteration):
    rvals_null = []
    for sub in range(nsubj):
        if np.all(np.isnan(thought_sim[sub])):
            r_sub = np.nan
        else:
            random.shuffle(thought_sim[sub])
            r_sub = nanspearmanr(fc_sim[sub],thought_sim[sub])
        rvals_null.append(r_sub)
    null_r.append(conv_z2r(np.nanmean(conv_r2z(np.array(rvals_null)))))
null_r = np.asarray(null_r)
p = onetail_p(actual_r,null_r)
print('r = ',actual_r,'p = ',p)

scipy.io.savemat('./results/RSA/content_within-sub.mat',{'actual_r':actual_r,'null_r':null_r})

0 among 10000 null has higher r value than actual prediction
r =  0.06940962247997925 p =  9.999000099990002e-05


# thought dimensions

In [3]:
df_dimension = pd.read_csv('./data/beh/all_ratings.csv')
# create a behavioral participant list
sub_list = np.unique(df_dimension['Sub'])
print('We have '+str(len(sub_list)) + ' participants')

We have 60 participants


In [4]:
var_list = ['Awake','External','Future','Past','Other','Valence','Image','Word','Detail']
all_dims = []
for var in var_list:
    this_dim = df_dimension[var]
    vals = []
    for row in range(len(this_dim)):
        vals.append(np.array(get_num(this_dim[row])))
    all_dims.append(np.array(vals))
all_dims = np.array(all_dims)

# make the 111th row - sub 1032 s1r1 - all nans due to response box error
all_dims = all_dims.astype(float)
all_dims[:,110,:] = np.nan

zscored_within_dim = []
for dim in range(len(var_list)):
    tmp_dim = stats.zscore(all_dims[dim].flatten(), nan_policy='omit').reshape(all_dims[dim].shape)
    zscored_within_dim.append(tmp_dim)
zscored_within_dim = np.array(zscored_within_dim)

In [5]:
nsubj = len(sub_list)
avg_dimensions = []
for sub in sub_list:
    this_index = df_dimension[df_dimension['Sub']==sub]['Run'].index
    tmp = zscored_within_dim[:,this_index,:]
    tmp1 = np.mean(tmp,axis=2) #average across trials in a run
    tmp2 = np.mean(tmp1,axis=1) #average across runs
    avg_dimensions.append(tmp2)

dimension_sim = [] # calculate between subject similarity
for i in range(nsubj-1):
    for j in range(i+1,nsubj):
        if np.any(np.isnan(avg_dimensions[i])) or np.any(np.isnan(avg_dimensions[j])):
            tmp_r = np.nan
        else:
            tmp_r = stats.spearmanr(avg_dimensions[i],avg_dimensions[j])[0]
        dimension_sim.append(tmp_r)
dimension_sim = np.asarray(dimension_sim)

In [13]:
# read in brain data
rest_fc = scipy.io.loadmat('./data/brain/rest_fc.mat')['rest'][0]
# calculate FC by subject
nsubj = len(rest_fc)
fc_bysub = []
for i in range(nsubj):
    fc_bysub.append(reshape_FC(rest_fc[i]))

fingerprint_bysub = []
for sub in range(nsubj):
    this_sub = np.nanmean(fc_bysub[sub],axis=1) # calculate the mean FC profile for each individual
    fingerprint_bysub.append(this_sub)

FP_sim = []
for i in range(nsubj-1):
    for j in range(i+1,nsubj):
        tmp_r = nancorr(fingerprint_bysub[i],fingerprint_bysub[j])
        FP_sim.append(tmp_r)
FP_sim = np.asarray(FP_sim)

/var/folders/dq/rwvdlsl16q96bmby7428s3700000gn/T/ipykernel_4510/789186149.py:11: RuntimeWarning: Mean of empty slice
  this_sub = np.nanmean(fc_bysub[sub],axis=1) # calculate the mean FC profile for each individual


In [14]:
actual_r = nanspearmanr(FP_sim,dimension_sim)
print(actual_r)
null_r = []
niteration = 10000
for iter in range(niteration):
    random.shuffle(dimension_sim)
    tmp_r = nanspearmanr(FP_sim,dimension_sim)
    null_r.append(tmp_r)
null_r = np.asarray(null_r)
p = onetail_p(actual_r,null_r)
print('r = ',actual_r,'p = ',p)

0.026085983454496448
1408 among 10000 null has higher r value than actual prediction
r =  0.026085983454496448 p =  0.14088591140885912


In [37]:
# read in brain data
rest_fc = scipy.io.loadmat('./data/brain/rest_fc.mat')['rest'][0]
# delete 1032 s1r1 (response box error), which is index 29 because there is no 1004 and 1019
rest_fc[29] = np.delete(rest_fc[29],[0],axis = 0) # we don't need to do this for content or topic because that's a problem with dimension rating

# calculate FC by subject
nsubj = len(rest_fc)
fc_bysub = []
for i in range(nsubj):
    fc_bysub.append(reshape_FC(rest_fc[i]))

df_dimension = pd.read_csv('./data/beh/all_ratings.csv')
# create a behavioral participant list
sub_list = np.unique(df_dimension['Sub'])
print('We have '+str(len(sub_list)) + ' participants')

# variable list
var_list = ['Awake','External','Future','Past','Other','Valence','Image','Word','Detail']

# create a behavioral dataset by subject
idx = -1
beh_bysub = []
for sub in range(len(sub_list)):
    beh_bysub_rating = []
    for var in range(len(var_list)):
        sub_data = df_dimension[df_dimension['Sub']==sub_list[sub]][var_list[var]]
        sub_vec = []
        for run in range(len(sub_data)):
            idx = idx + 1
            run_data = get_num(sub_data[idx])
            sub_vec.append(run_data)
        sub_vec = np.asarray(sub_vec)
        sub_vec = np.reshape(sub_vec,(sub_vec.shape[0]*sub_vec.shape[1]))
        # zscore within subject
        sub_vec = stats.zscore(sub_vec)
        beh_bysub_rating.append(sub_vec)
        if var == len(var_list) - 1:
            pass
        else:
            idx = idx - len(sub_data)
    beh_bysub_rating = np.asarray(beh_bysub_rating)
    beh_bysub_rating = np.transpose(beh_bysub_rating)
    beh_bysub.append(beh_bysub_rating)

# remove 1032 s1r1, 1044 s1r2
beh_bysub[29] = np.delete(beh_bysub[29],range(0,8),axis = 0)
beh_bysub[41] = np.delete(beh_bysub[41],range(8,16),axis = 0)

# let's double check the brain match behavior
count = 0
for i in range(nsubj):
    # print('sub',i,len(beh_bysub[i]) == fc_bysub[i].shape[1])
    if len(beh_bysub[i]) == fc_bysub[i].shape[1]:
        count = count + 1
if count == nsubj:
    print('All behavioral and brain data match')

We have 60 participants
All behavioral and brain data match


In [38]:
thought_sim = [] # calculate within subject between trial similarity
for i in range(nsubj):
    r_sub = []
    for j in range(beh_bysub[i].shape[0]-1):
        for k in range(j+1,beh_bysub[i].shape[0]):
            tmp_r = stats.spearmanr(beh_bysub[i][j,:],beh_bysub[i][k,:])[0]
            r_sub.append(tmp_r)
    thought_sim.append(np.array(r_sub))

# calculate FC similarity within subject
fc_sim = []
for i in range(nsubj):
    r_sub = []
    for j in range(fc_bysub[i].shape[1]-1):
        for k in range(j+1,fc_bysub[i].shape[1]):
            if np.sum(np.isnan(fc_bysub[i][:,j])) == 35778 or np.sum(np.isnan(fc_bysub[i][:,k])) == 35778:
                tmp_r = np.nan
            else:
                tmp_r = nancorr(fc_bysub[i][:,j],fc_bysub[i][:,k])
            r_sub.append(tmp_r)
    fc_sim.append(np.array(r_sub))

rvals = []
for sub in range(nsubj):
    if np.all(np.isnan(thought_sim[sub])):
        r_sub = np.nan
    else:
        r_sub = nanspearmanr(fc_sim[sub],thought_sim[sub])
    rvals.append(r_sub)

In [40]:
actual_r = conv_z2r(np.nanmean(conv_r2z(np.array(rvals))))
null_r = []
niteration = 10000
for iter in range(niteration):
    rvals_null = []
    for sub in range(nsubj):
        if np.all(np.isnan(thought_sim[sub])):
            r_sub = np.nan
        else:
            random.shuffle(thought_sim[sub])
            r_sub = nanspearmanr(fc_sim[sub],thought_sim[sub])
        rvals_null.append(r_sub)
    null_r.append(conv_z2r(np.nanmean(conv_r2z(np.array(rvals_null)))))
null_r = np.asarray(null_r)
p = onetail_p(actual_r,null_r)
print('r = ',actual_r,'p = ',p)

scipy.io.savemat('./results/RSA/dimension_within-sub.mat',{'actual_r':actual_r,'null_r':null_r})

0 among 10000 null has higher r value than actual prediction
r =  0.07143891008818332 p =  9.999000099990002e-05
